# Scrappring flight details from the website yatra.com

In [1]:
# Importing necessary library files
import selenium
import pandas as pd
import numpy as np
import time
import requests
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

In [3]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

In [4]:
# Getting the webpage of mentioned url
url = "https://www.yatra.com/"
driver.get(url)
time.sleep(3)

In [5]:
# Clicking on Search Flights button to navigate price of flight tickets
driver.find_element_by_xpath("/html/body/div[2]/div/section/div[1]/div/div[1]/section/div/div/div/div[1]/div[3]/div[2]/input").click()
time.sleep(3)

In [7]:
# Let's make a list of locations to scrap data
loclst = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai','Hyderabad ','Goa ','Kolkata ','Jaipur ','Lucknow ']

In [8]:
# Creating empty lists to fetch required data

Airline_name=[]      # The name of the airline.
Departure_time = []  # The time when the journey starts from the source.
Arrival_time = []    # Time of arrival at the destination.
Duration=[]          # Total duration taken by the flight to reach the destination from the source.
Stops = []           # Total stops between the source and destination.
Source = []          # The source from which the service begins.
Destination = []     # The destination where the service ends.
Meal=[]              # Availability of meals in the flight.
Price=[]             # The price of the flight ticket.
Location=[]          # The location of the flights 

In [9]:
# Scraping the data from the mentioned url

# Fetching webelement of source 
for x in loc_lst:
    for y in loc_lst:
        if x!=y:                 
            Source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(2)
            Source.clear() 
            Source.send_keys(x)
            time.sleep(2)
            
            # Fetching webelement of destination
            Destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            # Searching for flights again and clicking in search again button
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            # Scrolling the data 5000 times 
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
            
            # Fetching web element of scroll to top button
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
            
           # Fetching name of Airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    Airline_name.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching departure time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    Departure_time.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching arrival time of the flights 
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    Arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            
            # Fetching Duration of flights journey
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching number of stops the flights have between source and destination
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    Stops.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching availability of meal in flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    Meal.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching Locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching Prices of flight tickets
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    Price.append(i.text)
            except NoSuchElementException:
                pass    

In [10]:
Source = Location[0:len(Location):2]
Destination = Location[1:len(Location):2]

In [11]:
# Checking length of source and destination
len(Source), len(Destination)

(3627, 3627)

In [12]:
# Checking lengths of all features
print(len(Price),len(Airline_name),len(Departure_time),len(Arrival_time),len(Duration),len(Stops),len(Meal))

3627 3627 3627 3627 3627 3627 3627


In [13]:
# Creating a dataframe for scraped data
data = list(zip(Airline_name,Departure_time,Arrival_time,Duration,Source,Destination,Meal,Stops,Price))
df = pd.DataFrame(data, columns = ["Airline",
                                   "Departure_time",
                                   "Time_of_arrival",
                                   "Duration", 
                                   "Source",
                                   "Destination",
                                   "Meal_availability",
                                   "Number_of_stops",
                                   "Price"])
df

,Airline,Departure_time,Time_of_arrival,Duration,Source,Destination,Meal_availability,Number_of_stops,Price
0,Go First,08:45,10:55,2h 10m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
1,Go First,14:20,16:35,2h 15m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
2,Go First,21:00,23:15,2h 15m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
3,Go First,16:20,21:55,5h 35m,New Delhi,Mumbai,eCash 250,1 Stop,"5,954"
4,Go First,13:45,19:40,5h 55m,New Delhi,Mumbai,eCash 250,1 Stop,"5,954"
...,...,...,...,...,...,...,...,...,...
3622,Air India,20:15,19:10,22h 55m,Lucknow,Jaipur,Free Meal,1 Stop,"7,359"
3623,IndiGo,13:00,22:50,9h 50m,Lucknow,Jaipur,eCash 250,1 Stop,"8,912"
3624,Air India,11:05,13:15,26h 10m,Lucknow,Jaipur,Free Meal,1 Stop,"12,189"
3625,Air India,20:15,13:15,17h 00m,Lucknow,Jaipur,Free Meal,2 Stop(s),"13,977"


In [14]:
# Saving the data into excel file
df.to_excel("FlightPrice.xlsx")

I have successfully scraped the required data from the website yatra.com. The dataframe consists of 3627 rows and 9 columns. And I have saved this dataframe in excel format as "FlightPrices".